[![open in colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Oh-Taegyun/Pytorch_study/blob/main/part12_%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5%20%EC%A0%9C%EC%9E%91.ipynb)



### 0. 코랩에서 작업할때 쓰는 코드

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

!ls /content/gdrive/MyDrive/hymenoptera_data

SyntaxError: invalid syntax (1644683522.py, line 6)

### 1. 라이브러리 호출

In [6]:
import torch
import torch.nn as nn
import torch.autograd 
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms # 데이터 전처리를 위해 사용하는 라이브러리
import torchvision.models as models # 다양한 파이토치 네트워크를 사용할 수 있도록 도와주는 패키지
from torch.utils.data import DataLoader, Dataset

from torchvision import datasets

import matplotlib.pyplot as plt
import numpy as np
import time
import os

### 2. CPU인지 GPU인지 확인 후 device 설정

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #torch.cuda.is_available() GPU를 사용가능하면 True, 아니라면 False를 리턴

print("지금 사용하는 device :",device)

지금 사용하는 device : cuda:0


### 3. 데이터셋 설정

In [4]:
data_path = '/content/gdrive/MyDrive/Deep_Learning/Transfer_Learning/catanddog/train'

transforms = transforms.Compose(
    [
        transforms.Resize([256, 256]), # 이미지의 크기를 조정 256 x 256 크기로 이미지 데이터 조정
        transforms.RandomResizedCrop(224), # 이미지를 랜덤한 크기로 자름, 데이터의 확장 용도
        transforms.RandomHorizontalFlip(), # 이미지를 랜덤하게 수평으로 뒤집는다
        transforms.ToTensor() # 이미지 데이터를 텐서로 변환
    ]
)

train_dataset = torchvision.datasets.ImageFolder(data_path, transform=transforms)

train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)

print(len(train_dataset)) # train_dataset에 포함된 데이터의 개수를 출력해라

385


### 4. 사전 훈련된 모델 다운

In [8]:
resnet18 = models.resnet18(pretrained = True)

print(resnet18) # 여기서 출력된 fc가 Fully connected layer 로 완전연결 계층을 의미한다.

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### 5. 내려받은 모델에 대한 추가 설정
전이학습의 요점은 마지막 연결계층(Fully connected layer)만 학습하고 나머지 계층들은 학습이 안되도록 해야한다.  

이는 내려받은 모델의 파라미터를 고정해주지 않으면, 전이 학습 도중에 파라미터가 학습이 되어버리는 대참사가 일어나게된다.

따라서 파라미터에 대해서 학습이 되지 않도록 고정해야한다.

In [9]:
model = models.resnet18(pretrained=True) #resnet18을 그대로 불러와서

for param in model.parameters(): # model.parameters()은 모델의 파라미터를 반환한다.
    param.requires_grad = False # resnet18 모델에 더이상 역전파를 추적하지 않게 하여 모델의 합성곱층 가중치를 고정시킨다

model.fc = nn.Linear(512,2) # 완전연결층을 이제 수정해주자. 분류되는 클래스의 숫자는 2개이므로 출력을 2로 만든다. 원래 출력이 (fc): Linear(in_features=512, out_features=1000, bias=True) 이었는데 
# out_features=1000을 2로 바꿔주자
# __setattr__의 특성 덕에 이런 문장으로 새로운 계층이 추가가 가능하다.

for param in model.fc.parameters():
    param.requires_grad = True # 하지만 맨 마지막의 완전연결층은 계산 그래프를 형성하게 해서 역전파가 가능하게 함

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### 6. 모델 학습을 위한 함수 생성 

In [10]:
def train_model(model,dataloaders, crtierion, optimizer, device, save_path, num_epochs=25):
    since = time.time()
    acc_history = []
    loss_history = []
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        running_loss = 0.0 # 손실 함수
        running_corrects = 0 # 정답갯수

        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)
            model.to(device)

            optimizer.zero_grad() # 기울기를 0으로 설정
            outputs = model(inputs) # 순전파 학습
            loss = crtierion(outputs, labels) # 손실 함수 구하기
            _, preds = torch.max(outputs, 1) # 결과값 추출
            loss.backward() # 역전파 실행 이때 requires_grad = True가 된 완전연결층만 역전파가 됨
            optimizer.step() # 기울기 업데이트

            running_loss += loss.item() * inputs.size(0) # 출력 결과와 레이블의 오차를 계산한 결과를 누적하여 저장한다
            # loss.item() 으로 손실이 갖고 있는 스칼라 값을 가져올 수 있습니다.
            running_corrects += torch.sum(preds == labels.data) # 출력 결과와 레이블이 동일한지 확인한 결과를 누적하여 저장

        epoch_loss = running_loss / len(dataloaders.dataset)
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)

        print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss,epoch_acc)) # 참고) {:.소수점 자리수f} 포맷 코드 : {} 내에 실수의 소수점 자리수(.소수점 자리수f)를 지정해 줄 수 있음 소수점 4자리 까지 표시함
        if epoch_acc > best_acc: # 만약에 어떤 에폭에서의 정확도가 최고 정확도보다 높을 경우 업데이트
            best_acc = epoch_acc
        
        acc_history.append(epoch_acc.item())
        loss_history.append(epoch_loss)

        print()

    time_elapsed = time.time() - since # 실행 시간 계산
    print(f'실행 시간 : {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'이 모델의 최고 정확도: {best_acc:4f}') # 최고 정확도

    torch.save(model.state_dict(),save_path) # 모든 에폭을 소진할 때 모델의 상태를 저장 

    return acc_history, loss_history



### 7. 파라미터 학습 결과를 옵티마지어에 전달

In [ ]:
params_to_update = [] # 업데이트 할 파라미터를 저장하자
for name, param in model.named_parameters(): # 모델 에서 파라미터를 꺼내 오자
    if param.requires_grad == True: # 만일 역전파가 가능하도록 설정되어 있다면
        params_to_update.append(param) # 파라미터 학습 결과를 저장하자 (에초당시 resnet18에선 마지막 완전연결층만 역전파가 가능하도록 했으니 마지막의 fc 파아미터만 출력되어야 한다)
        print("\t", name)

### 8. 모델 학습

In [ ]:
optimizer = torch.optim.Adam(model.fc.parameters()) # 옵티마이저는 Adam으로 설정한다. 
cost = torch.nn.CrossEntropyLoss() # 손실 함수 정의 무난하게 교차 엔트로피 오차로 설정
criterion = nn.CrossEntropyLoss() # 손실함수지정
save_data_path = '/content/gdrive/MyDrive/Deep_Learning/Learned_Parameters/part12_model_state_dict.pt'
train_acc_hist, trian_loss_hist = train_model(model, train_loader, criterion, optimizer, device, save_data_path)

### 9. 모델 실전 테스트 전 이미지 전처리

In [ ]:
data_path = '/content/gdrive/MyDrive/Deep_Learning/Transfer_Learning/catanddog/test'

transforms = transforms.Compose(
    [
        transforms.Resize(224), # 이미지의 크기를 조정 224 x 224 크기로 이미지 데이터 조정
        transforms.CenterCrop(224), # 중앙을 기준으로 이미지를 자릅니다
        transforms.ToTensor() # 이미지 데이터를 텐서로 변환
    ]
)

test_dataset = torchvision.datasets.ImageFolder(data_path, transform=transforms)

test_loader = DataLoader(train_dataset,batch_size=32,num_workers=1,shuffle=True)

print(len(test_dataset)) # train_dataset에 포함된 데이터의 개수를 출력해라

### 10. 테스트 데이터 평가 함수

In [ ]:
def test_model(model,dataloaders,device,save_path):
    since = time.time()
    acc_history = []
    best_acc = 0.0

    model.load_state_dict(torch.load(save_path))
    model.eval() # 모델을 train에서 evaluation으로 변경 https://bluehorn07.github.io/2021/02/27/model-eval-and-train.html
    model.to(device)
    running_corrects = 0

    for inputs, labels in dataloaders:
        inputs, labels = inputs.to(device), labels.to(device)

        with torch.no_grad(): # 자동미분을 사용하지 않음 (학습 목적이 아닌 테스트 목적)
            outputs = model(inputs)

            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
            
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)
        print('Acc: {:.4f}'.format(epoch_acc))

        if epoch_acc > best_acc: # 만약에 어떤 에폭에서의 정확도가 최고 정확도보다 높을 경우 업데이트
            best_acc = epoch_acc
        
        acc_history.append(epoch_acc.item())
        print()
    
    time_elapsed = time.time() - since # 실행 시간 계산
    print(f'Validation complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}') # 최고 정확도

    return acc_history

### 11. 검증

In [ ]:
val_acc_hist= test_model(model, test_loader, device, save_data_path)

### 12. 모델의 훈련, 테스트 데이터의 정확도를 그래프로 확인 

In [ ]:
plt.plot(train_acc_hist)
plt.plot(val_acc_hist)
plt.show()

### 13. 오차 정보 그래프 확인 

In [ ]:
plt.plot(trian_loss_hist)
plt.show()

### 14. 예측 이미지 출력을 위한 전처리 함수

In [ ]:
def im_convert(tensor):  
    image=tensor.to("cpu").clone().detach().numpy()  
    image=image.transpose(1,2,0)  
    image=image*(np.array((0.5,0.5,0.5))+np.array((0.5,0.5,0.5)))  
    image=image.clip(0,1)  
    return image  

classes = {0:'cat', 1:'dog'} # 개와 고양이 두 개에 대한 레이블

dataiter = iter(test_loader) # 테스트 데이터 로더의 반복자 반환
images, labels = dataiter.next()  # 테스트 데이터 순차적으로 출력
images, labels = images.to(device), labels.to(device)

output = model(images)  

_,preds=torch.max(output,1) 

fig = plt.figure(figsize = (25,4))  

for idx in np.arange(20):  
    ax=fig.add_subplot(2,10,idx+1,xticks=[],yticks=[])  
    plt.imshow(im_convert(images[idx]))  
    ax.set_title(classes[labels[idx].item()])
    ax.set_title("{}({})".format(str(classes[preds[idx].item()]),str(classes[labels[idx].item()])),color=("green" if preds[idx]==labels[idx] else "red"))  

plt.show()  
plt.subplots_adjust(bottom=0.2, top=0.6, hspace=0)